# Measuring Catchment-RTT by ASN variation by Day in Brazil Node



In [ ]:
#Installing Software
#!pip install --upgrade google-cloud-bigquery[bqstorage,pandas,pandas-gbq,ipython,pyarrow]
#!pip install modin

### Loading Libraries (BQ, Datalab)


In [ ]:
# Checking current project on Google Cloud 
from google.datalab import Context

context = Context.default()
print('The current project is %s' % context.project_id)

In [ ]:
# Use IPython Magics for better performance
%load_ext google.datalab.kernel
%load_ext google.cloud.bigquery

In [ ]:
#Acess Database on BQ
#from google.cloud import bigquery
# https://cloud.google.com/resource-manager/docs/creating-managing-projects
#project_id = 'vp-measurements-testbed'
#client = bigquery.Client(project=project_id)
#
#for dataset in client.list_datasets():
#  print(dataset.dataset_id)

### Testing BQ with a direct result without dataframe

In [ ]:
%%bq query
SELECT
  COUNT (*) as TOTAL
FROM 
  `vp-measurements-testbed.datasetTestbed.measurements`

In [ ]:
from pandas_gbq import read_gbq


In [ ]:
projectid = 'vp-measurements-testbed'
queryid = """
select 
  count (*) as TOTAL 
from 
  `vp-measurements-testbed.datasetTestbed.measurements`
"""


In [ ]:

read_gbq(query=queryid, project_id=projectid, dialect='standard', verbose=True)

In [ ]:
QUERY = """
SELECT
  timestamp,
  catchment,
  asn,
  src_net,
  origin,
  bgp,
  id,
  rtt
FROM
  `vp-measurements-testbed.datasetTestbed.measurements`
WHERE
  catchment IN ('br-gru-anycast01')
ORDER BY
  timestamp ASC
LIMIT
 1000
 """

In [ ]:
!date

In [ ]:
read_gbq(query=QUERY, project_id=projectid, dialect='standard', verbose=True)

## Performance text with MODIN

In [ ]:

sql = """
  SELECT
    timestamp,
    catchment,
    asn,
    src_net,
    origin,
    bgp,
    id,
    rtt
  FROM
    `vp-measurements-testbed.datasetTestbed.measurements`
  WHERE
    catchment IN ('br-gru-anycast01')
  ORDER BY
    timestamp ASC
  LIMIT
    100
"""


In [ ]:
import modin.pandas as pd 
import numpy as np

In [ ]:
frame_data = np.random.randint(0, 100, size=(2**10, 2**8))
df = pd.DataFrame(frame_data)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
pd.read_gbq(sql, dialect='standard')

In [ ]:

# Run a Standard SQL query using the environment's default project
df = pandas.read_gbq(sql, dialect='standard')

## Performance text with google-cloud-bigquery

In [ ]:
from google.cloud import bigquery

client = bigquery.Client() 
sql = """
  SELECT
    timestamp,
    catchment,
    asn,
    src_net,
    origin,
    bgp,
    id,
    rtt
  FROM
    `vp-measurements-testbed.datasetTestbed.measurements`+
  WHERE
    catchment IN ('br-gru-anycast01')
  ORDER BY
    timestamp ASC
  LIMIT
    500000
"""


In [ ]:

# Run a Standard SQL query using the environment's default project
df = client.query(sql).to_dataframe()


In [ ]:
df.shape

In [ ]:
df.head()

### First look at data: (timestamp, catchment=brazil, asn, src_net, origin, bgp, id, rtt)


In [ ]:
# Executes BigQuery QUERY
QUERY = """
SELECT
  timestamp,
  catchment,
  asn,
  src_net,
  origin,
  bgp,
  id,
  rtt
FROM
  `vp-measurements-testbed.datasetTestbed.measurements`
WHERE
  catchment IN ('br-gru-anycast01')
ORDER BY
  timestamp ASC
LIMIT
 100000
 """

In [ ]:
# Load Query --> Dataframe
query_job = client.query(QUERY)
df = query_job.to_dataframe()

In [ ]:
df.head()

In [ ]:
len (df)

### Consider just ONE DAY,  same ORIGIN, same CATCHMENT (timestamp="2019-07-24", catchment=Brazil and origin-Brazil)


In [ ]:
QUERY = """
SELECT
  timestamp,
  catchment,
  asn,
  src_net,
  origin,
  bgp,
  id,
  rtt
FROM
  `vp-measurements-testbed.datasetTestbed.measurements`
WHERE
  catchment IN ('br-gru-anycast01') 
  AND ( DATE (timestamp) ='2019-07-24' )

#GROUP BY
# catchment,

ORDER BY
  timestamp ASC

LIMIT
 1000;
"""

In [ ]:
# Load Query --> Dataframe
query_job = client.query(QUERY)
df = query_job.to_dataframe()

In [ ]:
#%load_ext google.colab.data_table


In [ ]:
df


### rtt( min, max, avg, mean) in each asn - ASN x RTT



In [ ]:
#%%bigquery --project vp-measurements-testbed df
QUERY = """
SELECT
  timestamp,
  catchment,
  asn,
  src_net,
  origin,
  bgp,
  id,
  rtt
FROM
  `vp-measurements-testbed.datasetTestbed.measurements`
WHERE
  #catchment IN ('br-gru-anycast01') 
  #AND ( DATE (timestamp) ='2019-07-24' )
    ( DATE (timestamp) ='2019-07-24' )
LIMIT 600000
"""

In [ ]:
# Load Query --> Dataframe
query_job = client.query(QUERY)
df = query_job.to_dataframe()

In [ ]:
df.head()


In [ ]:
len(df)

# Hiden Things





In [ ]:
df

### Graphs


In [ ]:
import altair as alt
alt.Chart(df).mark_line().encode(
  x='t0_timestamp',
  y='t0_qt',
  color='catchment'
).interactive(bind_y=True)

In [ ]:
%%bigquery --project vp-measurements-testbed df
SELECT
  timestamp as t0
  FORMAT_TIMESTAMP('%Y%m%d', timestamp) AS t0_date,
  FORMAT_TIMESTAMP('%H', timestamp) AS t0_hour,
  FORMAT_TIMESTAMP('%H', timestamp) AS t0_min,
  catchment,
  COUNT(*) AS t0_qt
FROM
  `vp-measurements-testbed.datasetTestbed.measurements`
WHERE
  catchment IN ('jp-hnd-anycast01')
GROUP BY
  t0,
  t0_date,
  t0_hour,
  t0_min,
  catchment
ORDER BY
 t0_date,t0_hour,t0_min ASC
LIMIT 100